In [1]:
from pprint import pprint
import angr

In [2]:
project = angr.Project("fauxware")

In [17]:
cfg = project.analyses.CFG(collect_data_references = True)

WARNING | 2017-01-17 13:25:21,269 | angr.analyses.cfg_fast | "auto_load_libs" is enabled. With libraries loaded in project, CFGFast will cover libraries, which may take significantly more time than expected. You may reload the binary with "auto_load_libs" disabled, or specify "start" and "end" paramenters to limit the scope of CFG recovery.


In [18]:
all_nodes = cfg.nodes()
print "There are %d nodes" %len(all_nodes)
print "The first five nodes are"
pprint(all_nodes)

There are 92 nodes
The first five nodes are
[<CFGNode 0x4005ab (1) [0]>,
 <CFGNode _fini+0x9 (0x4008c1 5) [0]>,
 <CFGNode authenticate (0x400664 42) [0]>,
 <CFGNode __do_global_dtors_aux+0x32 (0x400602 6) [0]>,
 <CFGNode accepted (0x4006ed 14) [0]>,
 <CFGNode frame_dummy+0x20 (0x400660 2) [0]>,
 <CFGNode __do_global_ctors_aux+0x26 (0x4008a6 9) [0]>,
 <CFGNode __libc_csu_init+0x45 (0x400825 5) [0]>,
 <CFGNode main+0xa0 (0x4007bd 10) [0]>,
 <CFGNode 0x400530 (6) [0]>,
 <CFGNode authenticate+0x2e (0x400692 7) [0]>,
 <CFGNode call_gmon_start+0x10 (0x4005bc 2) [0]>,
 <CFGNode frame_dummy+0x18 (0x400658 8) [0]>,
 <CFGNode main+0xac (0x4007c9 10) [0]>,
 <CFGNode main+0xb6 (0x4007d3 2) [0]>,
 <CFGNode 0x4005c3 (13) [0]>,
 <CFGNode 0x400566 (10) [0]>,
 <CFGNode _fini (0x4008b8 9) [0]>,
 <CFGNode authenticate+0x2a (0x40068e 4) [0]>,
 <CFGNode __do_global_ctors_aux (0x400880 22) [0]>,
 <CFGNode 0x400872 (14) [0]>,
 <CFGNode 0x400516 (10) [0]>,
 <CFGNode _init+0xe (0x4004ee 5) [0]>,
 <CFGNode __li

NOTE TO SELF : what do these things mean?
What is a fake return?
Let's look at the first node of the main function, which is 0x40071d

predecessors and successors

In [19]:
main_first_node=  cfg.get_node(0x40071d)
print "Node 0x40071d: %s" %main_first_node


Node 0x40071d: <CFGNode main (0x40071d 33) [0]>


In [20]:
successors = cfg.get_successors(main_first_node, excluding_fakeret=False)
print "All successors to node %s are : " %main_first_node
pprint(successors)

All successors to node <CFGNode main (0x40071d 33) [0]> are : 
[<CFGNode main+0x21 (0x40073e 22) [0]>, <CFGNode 0x400510 (6) [0]>]


In [21]:
successors_2 = main_first_node.successors
pprint(successors_2)

[<CFGNode 0x400510 (6) [0]>]


In [22]:
# INDIRECT JUMPS
indirect_jumps = cfg.indirect_jumps
print("Here are all indirect jumps from the binary")
pprint (indirect_jumps)

# RECOVER MEMORY DATA
memory_data = cfg.memory_data
print("Here are all recovered memory data from the binary")
pprint(memory_data)
# MAPPING BETWEEN INSTRUCTINS AND MEMORY 
ins_to_memdata = cfg._insn_addr_to_memory_data
print("Here is a mapping between instructin address and memory data")
pprint(ins_to_memdata)

Here are all indirect jumps from the binary
{4195584: <IndirectJump 0x400500 - ins 0x400506>,
 4195596: <IndirectJump 0x40050c - ins 0x400510>,
 4195600L: <IndirectJump 0x400510 - ins 0x400510>,
 4195616L: <IndirectJump 0x400520 - ins 0x400520>,
 4195632L: <IndirectJump 0x400530 - ins 0x400530>,
 4195648L: <IndirectJump 0x400540 - ins 0x400540>,
 4195664L: <IndirectJump 0x400550 - ins 0x400550>,
 4195680L: <IndirectJump 0x400560 - ins 0x400560>,
 4195696L: <IndirectJump 0x400570 - ins 0x400570>,
 4195772L: <IndirectJump 0x4005bc - ins 0x4005bc>,
 4195842L: <IndirectJump 0x400602 - ins 0x400613>,
 4195848L: <IndirectJump 0x400608 - ins 0x400613>,
 4195928L: <IndirectJump 0x400658 - ins 0x40065e>,
 4196394L: <IndirectJump 0x40082a - ins 0x400839>,
 4196400L: <IndirectJump 0x400830 - ins 0x400839>,
 4196502L: <IndirectJump 0x400896 - ins 0x4008a4>,
 4196512L: <IndirectJump 0x4008a0 - ins 0x4008a4>}
Here are all recovered memory data from the binary
{4194872: \0x400238, 28 bytes, string/,


In [28]:
cfg2 = project.analyses.CFGAccurate(keep_state=True)
if cfg2 is not None:
    all_nodes = cfg2.nodes()
    for n in all_nodes:
        print("%s:\t\tstate %s, eax %s, ecx %s" % (n.name, n.input_state, n.input_state.regs.eax, n.input_state.regs.ecx))

authenticate+0x35:		state <simuvex.s_state.SimState object at 0x7f98b9cd00a0>, eax <BV32 Reverse(Reverse(strncmp_ret_823_64)[63:32])>, ecx <BV32 0x4007e0>
read:		state <simuvex.s_state.SimState object at 0x7f98ba558b40>, eax <BV32 0x3>, ecx <BV32 0xfffeff08>
__libc_csu_init+0x50:		state <simuvex.s_state.SimState object at 0x7f98b9b69410>, eax <BV32 0xffffffff>, ecx <BV32 0x4007e0>
main+0x37:		state <simuvex.s_state.SimState object at 0x7f98b9c28190>, eax <BV32 0x8>, ecx <BV32 0x4007e0>
None:		state <simuvex.s_state.SimState object at 0x7f98bb3e9f50>, eax <BV32 0x0>, ecx <BV32 0x4007e0>
rejected:		state <simuvex.s_state.SimState object at 0x7f98bb5fce10>, eax <BV32 0x0>, ecx <BV32 0x4007e0>
None:		state <simuvex.s_state.SimState object at 0x7f98ba49d5a0>, eax <BV32 0x1>, ecx <BV32 0x4007e0>
accepted+0xe:		state <simuvex.s_state.SimState object at 0x7f98baefff50>, eax <BV32 Reverse(Reverse(puts_828_64)[63:32])>, ecx <BV32 0x4007e0>
call_gmon_start:		state <simuvex.s_state.SimState object

In [30]:
main_func = project.kb.functions.function(name='main')

In [32]:
vfg = project.analyses.VFG(start=main_func.addr, context_sensitivity_level=3,interfunction_level=3)
pprint (vfg.graph.nodes())

WARNING | 2017-01-17 13:44:43,915 | angr.call_stack | Returning to an unexpected address 0x0
WARNING | 2017-01-17 13:44:43,989 | angr.call_stack | Returning to an unexpected address 0x0
WARNING | 2017-01-17 13:44:44,249 | angr.analyses.vfg | The given CFG is not normalized, which might impact the performance/accuracy of the VFG analysis.
WARNING | 2017-01-17 13:44:44,288 | simuvex.plugins.symbolic_memory | Concretizing symbolic length. Much sad; think about implementing.
WARNING | 2017-01-17 13:44:44,362 | simuvex.plugins.symbolic_memory | Concretizing symbolic length. Much sad; think about implementing.
WARNING | 2017-01-17 13:44:44,646 | claripy.vsa.strided_interval | Reversing a real strided-interval <128>0x1[0x0, 0xffffffffffffffff00ffffffffffffff](uninit) is bad
WARNING | 2017-01-17 13:44:44,649 | claripy.vsa.strided_interval | Reversing a real strided-interval <192>0x1[0x0, 0xffffffffffffff00ffffffffffffffff]R(uninit) is bad
WARNING | 2017-01-17 13:44:44,655 | claripy.vsa.strided

[VFGNode[0x400550] <<BlockID 0x400550 (None@0x40071dL -> 0x4007a0L@0x400664L -> 0x4006c8L@0x400550L) % normal>>,
 VFGNode[0x4000010] <<BlockID 0x4000010 (None@None -> None@0x40071dL -> 0x40078aL@0x400530L) % normal>>,
 VFGNode[0x4000000] <<BlockID 0x4000000 (None@0x40071dL -> 0x4007bdL@0x4006edL -> 0x4006edL@0x400510L) % normal>>,
 VFGNode[0x400570] <<BlockID 0x400570 (None@0x40071dL -> 0x4007c9L@0x4006fdL -> 0x400713L@0x400570L) % normal>>,
 VFGNode[0x4000050] <<BlockID 0x4000050 (None@0x40071dL -> 0x4007c9L@0x4006fdL -> 0x400713L@0x400570L) % exit>>,
 VFGNode[0x4000040] <<BlockID 0x4000040 (None@0x40071dL -> 0x4007c9L@0x4006fdL -> 0x4006fdL@0x400520L) % normal>>,
 VFGNode[0x400550] <<BlockID 0x400550 (None@0x40071dL -> 0x4007a0L@0x400664L -> 0x400664L@0x400550L) % normal>>,
 VFGNode[0x400713] <<BlockID 0x400713 (None@None -> None@0x40071dL -> 0x4007c9L@0x4006fdL) % normal>>,
 VFGNode[0x4000000] <<BlockID 0x4000000 (None@None -> None@0x40071dL -> 0x40076aL@0x400510L) % normal>>,
 VFGN

In [35]:
main_func.endpoints

[<BlockNode at 0x4007d3 (size 2)>]

In [38]:
last_node = vfg.get_any_node(0x4007d3)
last_state = last_node.state

In [40]:
memory = last_state.memory # Now what can we do with this?

In [41]:
regions = memory.regions
pprint(regions)

{'global': <simuvex.plugins.abstract_memory.MemoryRegion object at 0x7f98b8396990>,
 'stack_0x400664': <simuvex.plugins.abstract_memory.MemoryRegion object at 0x7f98b8396450>,
 'stack_0x4006ed': <simuvex.plugins.abstract_memory.MemoryRegion object at 0x7f98b8396a90>,
 'stack_0x40071d': <simuvex.plugins.abstract_memory.MemoryRegion object at 0x7f98b8396bd0>}


In [44]:
if regions is not None:
    # WRITEME: Now we can have a look at the abstract locations (alocs) of the main function's stack region
    main_func_region = regions.get('stack_%#x' % main_func.addr)
    alocs = main_func_region.alocs

    print("Abstract locations of the main procedure are:")
    pprint(alocs)

    # WRITEME: Derive stack layout information from abstract locations
    # you may did a little bit into the source code SimuVEX and claripy to see what members an aloc has.
    # related code are abstract_memory.py in SimuVEX and the vsa subpackage in claripy.
    # by default, region.alocs is a dict mapping (block address, statement ID) to a list of memory targets.
    # what we want is a list of stack offset and size of the corresponding memory access
    # let's do it here

    stack_layout = defaultdict(set)  # map offset to size
    for aloc in alocs.values():
        for segment in aloc._segment_list:
            stack_layout[segment.offset].add(segment.size)

    print("The stack layout looks like:")
    for offset in sorted(stack_layout.keys(), reverse=True):
        print("%#x %s" % (offset, stack_layout[offset]))

Abstract locations of the main procedure are:
{4195712: (400580h, 0) [],
 4196125L: (40071dh, 5) [Seg (-0x8L [ 8 ])],
 4196133L: (40071dh, 19) [Seg (-0x3cL [ 4 ])],
 4196136L: (40071dh, 24) [Seg (-0x48L [ 8 ])],
 4196140L: (40071dh, 28) [Seg (-0x10L [ 1 ])],
 4196144L: (40071dh, 32) [Seg (-0x20L [ 1 ])],
 4196153L: (40071dh, 39) [Seg (-0x50L [ 8 ])],
 4196175L: (40073eh, 15) [Seg (-0x50L [ 8 ])],
 4196197L: (400754h, 15) [Seg (-0x50L [ 8 ])],
 4196207L: (40076ah, 7) [Seg (-0x50L [ 8 ])],
 4196229L: (400774h, 15) [Seg (-0x50L [ 8 ])],
 4196251L: (40078ah, 15) [Seg (-0x50L [ 8 ])],
 4196270L: (4007a0h, 16) [Seg (-0x50L [ 8 ])],
 4196275L: (4007b3h, 6) [Seg (-0x2cL [ 4 ])],
 4196290L: (4007bdh, 7) [Seg (-0x50L [ 8 ])],
 67108880L: (4000010h, -1) [Seg (-0x2cL [ 1 ]), Seg (-0x28L [ 8 ]), Seg (-0x18L [ 8 ])]}


TypeError: 'type' object is not iterable